In [1]:
import json
import logging
import sys
import time
import csv
from datetime import date, datetime
from json import JSONDecodeError
from lxml import html as htmlRenderer
import requests

In [2]:
urlInfoComments = "https://comments.eu1.gigya.com/comments.getStreamInfo"
urlGetComments = "https://comments.eu1.gigya.com/comments.getComments"

In [9]:
def extractCommentFromHtml(renderedPageHtml=None, url=None):
    #       extract comment from html page
    user = renderedPage.xpath("//aside[@id='ancla_comentarios']//div[@class='autor ']/text()")[0]
    hora = renderedPage.xpath("//aside[@id='ancla_comentarios']//span[@class='hora']/text()")[0]
    fecha = renderedPage.xpath("//aside[@id='ancla_comentarios']//span[@class='fecha']/text()")[0]
    comentario = renderedPage.xpath("//aside[@id='ancla_comentarios']//div[@class='texto-comentario']//p/text()")[0]
    user = user.replace("\n", "")
    hora = hora.replace("horas", "").strip()
    parsedComment = {
        "urlNoticia": url,
        "fecha": fecha,
        "hora": hora,
        "user": user,
        "commentario": comentario,
        "order": 1
    }
    return [parsedComment]

In [10]:
def extractComments(commentsList=None, urlNoticia=None, specialCase=None):
    logging.debug(" \t -> parsing comments list with -{}- elements:".format(len(commentsList)))
    parsedComments = []
    listToParse = commentsList[1] if specialCase else commentsList
    for commentObj in listToParse:
        fechaObj = datetime.fromtimestamp(round(commentObj["timestamp"] / 1000))
        fechaStr = fechaObj.strftime("%d/%m/%Y-%H:%M:%S")
        fechaArr = fechaStr.split("-")
        parsedComment = {
            "urlNoticia": urlNoticia,
            "fecha": fechaArr[0],
            "hora": fechaArr[1],
            "user": commentObj['sender']['name'],
            "commentario": commentObj['commentText'],
            "negVotes": commentObj["TotalVotes"] - commentObj["posVotes"],
            "posVotes": commentObj["posVotes"]
        }
        parsedComments.append(parsedComment)
        if commentObj.get("replies") != None:
            parsedComments = parsedComments + extractComments(commentObj.get("replies"), urlNoticia)
    return parsedComments

In [12]:
def lookupForComments(renderedPageHtml=None, url=None):
    streamIDStr = ""

    pageComments = []
    if ".es/noticia/" in url:
        streamIDStr = "noticia-{}".format(url.split("/noticia/")[1].split("/")[0])
    elif "/noticia/" in url:
        auxSplitted = url.split("/noticia/")[1].split("/")[0].split("-")
        streamIDStr = "noticia-{}".format(auxSplitted[len(auxSplitted) - 1])

    if (streamIDStr != ""):
        commentElValStreamId = streamIDStr
        logging.debug(" \t-> comment-stream-id to get comments is: {}".format(commentElValStreamId))

        infoArg = {
            "categoryID": "prod",
            "streamID": commentElValStreamId,
            "ctag": "comments_v2",
            "APIKey": "3_Hob3SXLWsBRwjPFz4f4ZEuxPo2PyeElFMWgWO2EXdBYUqwMk8lxNj78Fi_2VW5cH",
            "sourceData": {"categoryID": "prod", "streamID": commentElValStreamId},
            "pageURL": url,
            "cid": "",
            "source": "showCommentsUI",
            "sdk": "js_latest",
            "authMode": "cookie",
            "format": "json",
            "callback": "gigya.callback"
        }
        responseInfoComments = requests.get(urlInfoComments, infoArg)
        print(" -> info url: {}".format(responseInfoComments.url))
        infoComments = json.loads(responseInfoComments.text)
        logging.debug(" \t-> getting information for article: {}".format(url))
        logging.debug(
            " \t-> total of comments for current article: {}".format(infoComments["streamInfo"]["commentCount"]))
        if infoComments["streamInfo"]["commentCount"] > 0:
            # La info dice que hay comentarios, se procede a obtenerlos
            logging.debug(" -> total of comments is greater than 0")
            nextTs = ""
            iterate = True
            while (iterate):
                commentsArgs = {
                    "categoryID": "prod",
                    "streamID": commentElValStreamId,
                    "sourceData": {"categoryID": "prod", "streamID": commentElValStreamId},
                    "pageURL": url,
                    "includeSettings": "true",
                    "threaded": "true",
                    "includeUserOptions": "true",
                    "includeUserHighlighting": "true",
                    "lang": "es",
                    "ctag": "comments_v2",
                    "APIKey": "3_Hob3SXLWsBRwjPFz4f4ZEuxPo2PyeElFMWgWO2EXdBYUqwMk8lxNj78Fi_2VW5cH",
                    "cid": "",
                    "source": "showCommentsUI",
                    "sdk": "js_latest",
                    "authMode": "cookie",
                    "format": "json",
                    "callback": "gigya.callback"
                }
                if nextTs != "":
                    commentsArgs.update({
                        "start": nextTs
                    })
                responseComments = requests.get(urlGetComments, commentsArgs)
                print(" -> comments iterating url: {}".format(responseComments.url))
                commentsResponse = json.loads(responseComments.text)
                iterate = commentsResponse["hasMore"]
                nextTs = commentsResponse["next"]
                pageComments = pageComments + extractComments(commentsResponse["comments"], url)
            logging.debug(" \t -> retrieved total of {} comments".format(len(pageComments)))
            logging.debug(
                "#############################################################################################")
    return pageComments


In [6]:
url = "https://www.20minutos.es/noticia/3663038/0/sanchezreunion-iglesias-rivera-casado/"
html = requests.get(url)

In [7]:

renderedPage = htmlRenderer.fromstring(html.text)
# user = renderedPage.xpath("//aside[@id='ancla_comentarios']//div[@class='autor ']/text()")[0]
# hora = renderedPage.xpath("//aside[@id='ancla_comentarios']//span[@class='hora']/text()")[0]
# fecha = renderedPage.xpath("//aside[@id='ancla_comentarios']//span[@class='fecha']/text()")[0]
# comentario = renderedPage.xpath("//aside[@id='ancla_comentarios']//div[@class='texto-comentario']//p/text()")[0]

In [13]:
comments = lookupForComments(renderedPage, url)

 -> info url: https://comments.eu1.gigya.com/comments.getStreamInfo?categoryID=prod&streamID=noticia-3663038&ctag=comments_v2&APIKey=3_Hob3SXLWsBRwjPFz4f4ZEuxPo2PyeElFMWgWO2EXdBYUqwMk8lxNj78Fi_2VW5cH&sourceData=categoryID&sourceData=streamID&pageURL=https%3A%2F%2Fwww.20minutos.es%2Fnoticia%2F3663038%2F0%2Fsanchezreunion-iglesias-rivera-casado%2F&cid=&source=showCommentsUI&sdk=js_latest&authMode=cookie&format=json&callback=gigya.callback
 -> comments iterating url: https://comments.eu1.gigya.com/comments.getComments?categoryID=prod&streamID=noticia-3663038&sourceData=categoryID&sourceData=streamID&pageURL=https%3A%2F%2Fwww.20minutos.es%2Fnoticia%2F3663038%2F0%2Fsanchezreunion-iglesias-rivera-casado%2F&includeSettings=true&threaded=true&includeUserOptions=true&includeUserHighlighting=true&lang=es&ctag=comments_v2&APIKey=3_Hob3SXLWsBRwjPFz4f4ZEuxPo2PyeElFMWgWO2EXdBYUqwMk8lxNj78Fi_2VW5cH&cid=&source=showCommentsUI&sdk=js_latest&authMode=cookie&format=json&callback=gigya.callback
 -> commen

In [56]:
comments

[{'commentario': '¿ disculpas ? por aquí que no vengan , no queremos gentuza en España.',
  'fecha': '02/06/2019',
  'hora': '02:04',
  'order': 1,
  'urlNoticia': 'https://www.elmundo.es/espana/2019/06/01/5cf24d51fc6c8324098b458a.html',
  'user': 'ElCid'}]

In [34]:

el = renderedPage.find_class("js-ueCommentsCounter")
el_1 = el[0]
el_2 = el[1]

In [35]:
# print(el_1.get("data-commentid"))
# print(el_2.get("data-commentid"))

# print(el_1.get("data-commentid"))
# id_el_2.get("data-commentid"))

1605518
1605518


In [40]:
idNoticia=1605518
params = {"noticia": idNoticia, "version": "v2"}
response = requests.get(urlGetComments, params)
obj = json.loads(response.text, encoding="utf-8")

In [45]:
iterate = obj["lastPage"]
total = obj["total"]
print(iterate, total)

False 1126


In [46]:
firstComments = extractComments(obj['items'], url)

 	 -> parsing comments list with -10- elements:
 	 -> parsed comments returned with -10- elements:


In [47]:
firstComments

[{'commentario': '#1184 .. .. Jaime de Berenguer: El 28 de abril, en las elecciones generales podemos elegir entre; un traidor enfermo de ego (Pedro Sánchez) .. un tipo brillante en un partido sucio (Pablo Casado) .. un veleta que igual dice una cosa y la contraria (Albert Rivera) .. un lunático enfermo de odio enemigo de todo (Pablo Iglesias) .. un hombre valiente y recto que ama España (Santiago Abascal)',
  'fecha': '16/02/2019',
  'hora': '13:40',
  'order': 1191,
  'urlNoticia': 'https://www.elmundo.es/espana/2019/02/15/5c667ba8fc6c83bb478b45f6.html',
  'user': 'OBIS'},
 {'commentario': 'Santiago Abascal (VOX): La España Viva ha derrotado por fin una legislatura infame que empezó con un gobierno incapaz y cobarde, y termina con un gobierno ilegítimo y traidor . España, otra vez, ha sido más fuerte que sus enemigos .... El 28 de abril reconquistará su futuro. Juntos haremos historia',
  'fecha': '16/02/2019',
  'hora': '13:23',
  'order': 1190,
  'urlNoticia': 'https://www.elmundo.

In [48]:
nextComments = total - len(firstComments)
specialCase = False
if (nextComments == 1):
    print(" -> special case, adding extra value in order to avoid wrong behaviour")
    specialCase = True
    nextComments = nextComments + 1

In [49]:
nextComments

1116

In [50]:
params = {"noticia": idNoticia, "version": "v2", "pagina": nextComments}
response = requests.get(urlGetComments, params)

In [51]:
response.url

'https://www.elmundo.es/servicios/noticias/scroll/comentarios/comunidad/listar.html?noticia=1605518&version=v2&pagina=1116'

In [79]:
fileName = "test-elmundo-noticia-2.csv"

with open(fileName, "w") as file:
    csvwriter = csv.writer(file)
    count = 0
    for dataObj in comments:
        if count == 0:
            header = dataObj.keys()
            csvwriter.writerow(header)
            count += 1
        csvwriter.writerow(dataObj.values())


In [80]:
len(comments)

1126

In [81]:
import pandas as pd
pandas_obj = pd.read_csv(fileName, encoding="utf-8")
# filtered = pandas_obj.filter("urlNoticia=https://www.elmundo.es/espana/2019/02/15/5c667ba8fc6c83bb478b45f6.html")
# filtered.groupby("urlNoticia")["order"].nunique().to_frame()

In [82]:
pandas_obj

,urlNoticia,fecha,hora,user,commentario,order
0,https://www.elmundo.es/espana/2019/02/15/5c667...,16/02/2019,13:40,OBIS,#1184 .. .. Jaime de Berenguer: El 28 de abril...,1191
1,https://www.elmundo.es/espana/2019/02/15/5c667...,16/02/2019,13:23,OBIS,Santiago Abascal (VOX): La España Viva ha derr...,1190
2,https://www.elmundo.es/espana/2019/02/15/5c667...,16/02/2019,13:12,OBIS,@Ignatius_Mac #373 .. .. Un 11% de los votante...,1189
3,https://www.elmundo.es/espana/2019/02/15/5c667...,16/02/2019,13:12,OBIS,@Ignatius_Mac #373 .. .. El 15% del voto de VO...,1188
4,https://www.elmundo.es/espana/2019/02/15/5c667...,16/02/2019,13:04,OBIS,@ALMAGUGA #1083 .. .. Extrema izquierda: PODEM...,1187
5,https://www.elmundo.es/espana/2019/02/15/5c667...,16/02/2019,13:03,OBIS,@Ciudadanno #499 #487 .. .. El libro lo ha esc...,1186
6,https://www.elmundo.es/espana/2019/02/15/5c667...,16/02/2019,12:58,OBIS,@jrmb123 #1174 . #1178 #1181 #1184 .. .. Ahora...,1185
7,https://www.elmundo.es/espana/2019/02/15/5c667...,16/02/2019,12:28,OBIS,#1178 #1181 .. .. La humanidad se divide en in...,1184
8,https://www.elmundo.es/espana/2019/02/15/5c667...,16/02/2019,12:22,OBIS,"@carluqui #1060 #1181 #1182 .. .. Periodista, ...",1183
9,https://www.elmundo.es/espana/2019/02/15/5c667...,16/02/2019,12:13,OBIS,.@carluqui #1060 #1181 .. .. UPYD: Eliminando ...,1182


In [83]:
pandas_obj.count()

urlNoticia     1126
fecha          1126
hora           1126
user           1126
commentario    1126
order          1126
dtype: int64

In [84]:
duplicateRowsDF = pandas_obj[pandas_obj.duplicated()]

In [85]:
duplicateRowsDF.count()

urlNoticia     0
fecha          0
hora           0
user           0
commentario    0
order          0
dtype: int64

In [86]:
filtered = pandas_obj[(pandas_obj.urlNoticia == "https://www.elmundo.es/espana/2019/02/15/5c667ba8fc6c83bb478b45f6.html") & (pandas_obj.user == "carluqui")]

In [87]:
filtered

,urlNoticia,fecha,hora,user,commentario,order
23,https://www.elmundo.es/espana/2019/02/15/5c667...,16/02/2019,00:48,carluqui,"@trabajojm #1141 no amigo, no estoy diciendo e...",1168
119,https://www.elmundo.es/espana/2019/02/15/5c667...,15/02/2019,19:07,carluqui,PP-Ciudadanos-Vox-Merkel empiezan a frotarse l...,1060
